# RAG Exercise Solution

In [2]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [3]:
# We populated the RAG with the data from the data/calories.csv file in the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nuntrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [9]:
query = "What can a child eat if they need about 1500 calories a day?"
results = nuntrition_qna_db.query(query_texts=[query], n_results=3)

In [10]:
results

{'ids': [['qa_3974', 'qa_3892', 'qa_4720']],
 'embeddings': None,
 'documents': [["Question: What is the suggested daily calorie consumption for young babies aged between 0 and 6 months?\n        Answer: During their first six months of life, it's recommended that infants consume about 92 kcal/kg per day. This value takes into consideration factors like body size, physical activity levels and growth rates.\n\n        This Q&A pair provides information about nutrition and health topics.",
   "Question: What are the recommended food portions for a child aged 6-12 months?\n        Answer: An infant aged 6-12 months should consume around 30-45 grams of cereal, 15 grams of pulses, 200-500 milliliters of animal milk or top milk if breastfed, 50 grams of roots and tubers, 25 grams each of green leafy vegetables and other vegetables, 100 grams of fruit, 25 grams of sugar, and 10 grams of fat in a day's diet.\n\n        This Q&A pair provides information about nutrition and health topics.",
   

In [4]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)


@function_tool
def nutrtition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nuntrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [5]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrtition_qna_tool],
)

In [6]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are concise, evidence-aligned options for pregnant women and calories:

- Best meal components
  - Protein: lean meats, fish (low-mercury), eggs, beans, lentils, tofu
  - Whole grains: brown rice, oats, whole-wheat products
  - Dairy or fortified alternatives: milk, yogurt, cheese, fortified plant milks
  - Iron-rich foods: lean beef, poultry, beans, lentils, fortified cereals; pair with vitamin C-rich foods to enhance absorption
  - Folate-rich foods: leafy greens, citrus, beans, fortified grains
  - Calcium and vitamin D: dairy or fortified alternatives; safe sun exposure or supplements if needed
  - Healthy fats: avocado, nuts, seeds, olive oil; include omega-3 sources like fatty fish (low-mercury) or algae-based DHA
  - Hydration: plenty of water; limit caffeine

- Calorie guidance
  - Add roughly 200 extra calories per day via an extra small meal or snack.

- Practical meal ideas
  - Breakfast: fortified cereal with milk and berries; yogurt with nuts and fruit
  - Lunch: quin

In [12]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What is the best food for breakfast for a 6 year old child?",
    )
    print(result.final_output)

For a 6-year-old, aim for a balanced breakfast with protein, whole grains, fruit/vegetable, and dairy.

Good options:
- Oatmeal made with milk, topped with berries or sliced banana
- Scrambled eggs with whole-grain toast and a piece of fruit
- Yogurt (unsweetened) with granola and sliced fruit
- Whole-grain cereal with milk and a side of fruit
- Peanut butter on whole-wheat toast with a small glass of milk (if no peanut allergy)

Portion tips:
- Roughly 300–450 calories total, adjusted to the child’s appetite and activity level
- Include a protein source (eggs, yogurt, milk, cheese, nuts/butter if appropriate), a carbohydrate source (whole grains), and a fruit/vegetable

Want recipe ideas or calorie estimates for your child?
